In [1]:
import opticalflow3D

In [2]:
farneback = opticalflow3D.Farneback3D()

In [3]:
image2 = opticalflow3D.helpers.load_image([r"E:\data\3DTFM\test_set_2021Nov02_3c\2021Nov02_3DTFM_3_w1CSU RFP_t1.TIF",
                                           r"E:\data\3DTFM\test_set_2021Nov02_3c\2021Nov02_3DTFM_3_w1CSU RFP_t1-file002.TIF"])

In [4]:
image1 = opticalflow3D.helpers.load_image([r"E:\data\3DTFM\test_set_2021Nov02_3c\2021Nov02_3DTFM_3c_w1CSU RFP_t1.TIF",
                                           r"E:\data\3DTFM\test_set_2021Nov02_3c\2021Nov02_3DTFM_3c_w1CSU RFP_t1-file002.TIF"])

In [5]:
image2 = opticalflow3D.helpers.realign_image(image2, z_reverse=True, z_end=-2, y_end=-1, x_end=-1)
image1 = opticalflow3D.helpers.realign_image(image1, z_reverse=True, z_start=2, y_start=1, x_start=1)

In [ ]:
%%timeit
output_vz, output_vy, output_vx, output_error = farneback.calculate_flow(image1, image2, start_point=(0, 300, 90),
                                                                         total_vol=(998, 800, 1170),
                                                                         sub_volume=(350, 350, 350),
                                                                         overlap=(64, 64, 64),
                                                                         threadsperblock=(8, 8, 8),
                                                                        )

Running 3D Farneback optical flow with the following parameters:
Iters: 5 | Levels: 5 | Scale: 0.5 | Kernel: 7 | Filter: box-21 | Presmoothing: 7


Item: 1/12:   0%|                                                                                | 0/5 [00:00<?, ?it/s]C:\Users\Xianbin\anaconda3\envs\py39\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (27) < 2 * SM count (92) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
C:\Users\Xianbin\anaconda3\envs\py39\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (27) < 2 * SM count (92) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
C:\Users\Xianbin\anaconda3\envs\py39\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (27) < 2 * SM count (92) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
Item: 1/12:  20%|██████████████▍                                                         | 1/5 [00:05<00:23,  5.90s/it]C:\Users\Xianbin\anacond

Running 3D Farneback optical flow with the following parameters:
Iters: 5 | Levels: 5 | Scale: 0.5 | Kernel: 7 | Filter: box-21 | Presmoothing: 7


Item: 12/12: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.35s/it]


Running 3D Farneback optical flow with the following parameters:
Iters: 5 | Levels: 5 | Scale: 0.5 | Kernel: 7 | Filter: box-21 | Presmoothing: 7


...

Item: 12/12: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.35s/it]


Running 3D Farneback optical flow with the following parameters:
Iters: 5 | Levels: 5 | Scale: 0.5 | Kernel: 7 | Filter: box-21 | Presmoothing: 7


Item: 12/12: 100%|███████████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.36s/it]


Running 3D Farneback optical flow with the following parameters:
Iters: 5 | Levels: 5 | Scale: 0.5 | Kernel: 7 | Filter: box-21 | Presmoothing: 7


Item: 9/12:  80%|█████████████████████████████████████████████████████████▌              | 4/5 [00:11<00:02,  2.81s/it]

In [ ]:
opticalflow3D.helpers.save_flow("2021Nov02_3DTFM_3-5-5-0.5-7-21.npz", output_vz, output_vy, output_vx)
opticalflow3D.helpers.save_error("2021Nov02_3DTFM_3-5-5-0.5-7-21_error.npz", output_error)